In [1]:
import pandas as pd
import Bio as bio

In [2]:
df = pd.read_csv("in.txt", sep = '\t')

In [3]:
col1 = df["accesion"]
lista = list(col1)
lista

['ACL94627.1', 'BCM90541.1', 'BCM93164.1', 'AIR61364.1']

In [4]:
#BUSQUEDA PROTEICA - MODELADO BASE DE DATOS
from Bio import SeqIO, Entrez

def extract_protein_data(genbank_files):
    # Inicializar listas para almacenar los datos
    protein_data = []
    
    Entrez.email = "scminat@utn.edu.ec"   
    Entrez.api_key = "1e71a6fbed3171a40ea7a6cd9e0ed8298009"
    
    for genbank_id in genbank_files:
        # Fetch the GenBank record from the NCBI database
        handle = Entrez.efetch(db="protein", id=genbank_id, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        
        # Crear un diccionario para almacenar los datos de cada acceso
        accession_data = {}
        
        for feature in record.features:
            if feature.type == "CDS":
                # Extraer datos de cada característica
                protein_id = feature.qualifiers.get("protein_id", ["N/A"])[0]
                gene_name = feature.qualifiers.get("gene", ["N/A"])[0]
                locus_tag = feature.qualifiers.get("locus_tag", ["N/A"])[0]
                # Agregar los datos al diccionario
                accession_data["Gene Name"] = gene_name
                accession_data["Locus Tag"] = locus_tag
                
            elif feature.type == "source":
                # Obtener los datos de bioSample
                isolation_source = feature.qualifiers.get("isolation_source", ["N/A"])[0]
                lat_lon = feature.qualifiers.get("lat_lon", ["N/A"])[0]
                geo_loc_name = feature.qualifiers.get("geo_loc_name", ["N/A"])[0]
                host = feature.qualifiers.get('host', [""])[0]
                # Agregar los datos al diccionario
                accession_data["Isolation Source"] = isolation_source
                accession_data["Latitude"] = lat_lon
                accession_data["Longitude"] = lat_lon
                accession_data["Geo Location Name"] = geo_loc_name
                accession_data["Host"] = host
                        
        # Agregar el diccionario a la lista de datos
        protein_data.append(accession_data)
    
    # Crear un DataFrame combinado con todos los datos
    combined_df = pd.DataFrame(protein_data)
    
    # Exportar el DataFrame combinado a un archivo CSV
    combined_df.to_csv('out.csv', index=False)
    
    # Mostrar el DataFrame combinado
    print("Combined DataFrame:")
    print(combined_df)
    return combined_df


In [5]:
ej1 = lista
ej1

['ACL94627.1', 'BCM90541.1', 'BCM93164.1', 'AIR61364.1']

In [6]:
# Lista de IDs de GenBank
genbank_files = ej1

# Llamar a la función y almacenar el DataFrame combinado
combined_df = extract_protein_data(genbank_files)


Combined DataFrame:
        Isolation Source              Latitude             Longitude  \
0                    N/A                   N/A                   N/A   
1  bark of Acer palmatum  35.1073 N 139.0477 E  35.1073 N 139.0477 E   
2  bark of Acer palmatum  35.1073 N 139.0477 E  35.1073 N 139.0477 E   
3              waterfall                   N/A                   N/A   

                                   Geo Location Name           Host Gene Name  \
0                                                N/A                     bglX   
1  Japan:Shizuoka, Atami, The New Technology Deve...  Acer palmatum       N/A   
2  Japan:Shizuoka, Atami, The New Technology Deve...  Acer palmatum      lacG   
3                                           Malaysia                      N/A   

     Locus Tag  
0   CCNA_01162  
1  IAD21_02395  
2  IAD21_05052  
3   LH23_12085  
